# K means

## 导入数据

In [52]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder #将class(三种花的名字)编码为数值
import numpy as np

path = '/home/ysq/桌面/Iris/iris.data'

#读取数据
df = pd.read_csv(path,header=None)
df.head(5)

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## 为类别编码

In [53]:
#为类别编码,将花的名字换为数字
encoder = LabelEncoder().fit(df[4])
df[4] = encoder.transform(df[4])
df.head(5)

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


## 将dataframe转为ndarray

In [54]:
#将pandas读取的数据转为了ndarray型,(该数据中包含标签)
data = df.values
print(type(data))
print(data.shape)

<class 'numpy.ndarray'>
(150, 5)


## 定义聚类中心类,执行算法后生成的三个聚类中心向量
## 用这个类来储存

In [55]:
class Center:
    """
        聚类中心类，接受三个向量
    """
    def __init__(self,z0=None,z1=None,z2=None):
        """
            z0,1,2 = None可以使得在生成一个Center类时不必传入参数，之后可以进行传参初始化
        """
        self.z0 = z0
        self.z1 = z1
        self.z2 = z2

#### 初始化一个聚类中心类（可以选取data的前三个，亦可从每类中选一个，
#### Kmeans可能对初始化敏感，我们来验证他）

In [56]:
#定义一个接收聚类中心类的列表
center_list = []

c0 = Center()
#选取data中的前三个向量的前四个元素（省略标签），设置其为初始聚类中心
c0.z0 = data[0][0:4]
c0.z1 = data[1][0:4]
c0.z2 = data[2][0:4]

center_list.append(c0)#将初始化后的聚类中心传入列表储存

## 定义三个列表，用与接收分类器分类后的数据

In [57]:
list_class0 = []
list_class1 = []
list_class2 = []


### 全局函数1、计算两个向量之间的欧氏距离

In [58]:
def Euclid_distance(vector1,vector2):
    """
        计算向量1和向量2之间的欧氏距离
    """
    foo = vector1-vector2
    return np.linalg.norm(foo,ord=2)#利用了numpy线性代数内置函数计算L2范数

### 全局函数2、计算一个数据集中的中心向量（质心）

In [59]:
def center_of_set(data):
    """
        计算数据集的中心向量
    """
    #定义1*4的零向量
    x = np.zeros((1,4))
    for sample in data:
        x += sample[0:4]
    return x/data.shape[0]

# 定义Kmeans类，构建算法

In [60]:
class Kmeans_for_iris:
    #构造函数初始化
    def __init__(self,data,list0,list1,list2,center_list):
        """
            data是传入的数据集，list0，list1，list2是接收分类后数据的列表
            center_list是接收聚类中心的列表
        """
        self.data = data#数据
        self.list0 = list0#第一类数据
        self.list1 = list1#第二类数据
        self.list2 = list2#第三类数据
        self.center_list = center_list#聚类中心列表
        
    
    def compute_distance_from_three_center(self,i):
        """
            计算data中第i个样本与三个聚类中心的距离
            返回距离最近的一个聚类中心的序号（如：0,1,2）
        """
        #print(center_list[-1].z1[0:4])只取前四个元素参与计算
        #三个中心向量，取前四个元素运算，因为第五个是类别标签
        center0 = self.center_list[-1].z0[0:4]
        center1 = self.center_list[-1].z1[0:4]
        center2 = self.center_list[-1].z2[0:4]
        #取数据集中第i个元素的前4列，即其前四个元素
        foo = self.data[i][0:4]
        #计算距离
        dist0 = Euclid_distance(foo,center0)
        dist1 = Euclid_distance(foo,center1)
        dist2 = Euclid_distance(foo,center2)
        
        #print("data中第",i,"个数据与当前三个聚类中心的距离为：",dist0,' ',dist1,' ',dist2)    
        if dist0 <= dist1 and dist0 <= dist2:
            return 0
        if dist1 <= dist0 and dist1 <= dist2:
            return 1
        if dist2 <= dist0 and dist2 <= dist1:
            return 2
    
    def data_classify(self):
        """
            给数据分类，并初始化或更新分类后的数据列表list0,1,2
        """
        #print(data.shape[0])返回data矩阵的行数,也即该数据集有多少个样本
        
        #Warning:分类前现将接受分类样本的列表清空，不然列表会一直增长！
        self.list0.clear()
        self.list1.clear()
        self.list2.clear()
         
        for i in range(data.shape[0]):
            #类中函数互相调用有两种方法，此处调用计算距离的函数
            #Kmeans_for_iris.compute_distance_from_three_center(self,i)
            foo = self.compute_distance_from_three_center(i)#foo接收该样本被分到的类
            
            #Warning:分类前现将接受分类样本的列表清空，不然列表会一直增长！
            #根据foo的值对样本分类
            if foo == 0:
                self.list0.append(data[i])
            if foo == 1:
                self.list1.append(data[i])
            if foo == 2:
                self.list2.append(data[i])
        
    def compute_center_vector(self):
        """
            计算的聚类中心向量,更新center_list
        """
        #先定义一个Center类
        c = Center()
        #由于Center类中的成员z0,z1,z2储存的元素为ndarray型，需要对list0,1,2进行类型转换
        foo0 = np.array(self.list0)
        foo1 = np.array(self.list1)
        foo2 = np.array(self.list2)
        
        c.z0 = center_of_set(foo0)#计算第一类中心向量并赋值给Center类的成员z0
        c.z1 = center_of_set(foo1)
        c.z2 = center_of_set(foo2)
        #将新计算出的聚类中心加到列表中
        self.center_list.append(c)
    
    def stop_compute(self):
        """
            是否停止计算?
            如果center_list的倒数第一个元素和倒数第二个元素相等
            停止迭代，分类完成！
        """
        #当center_list仅有一个元素时（第0次计算），直接返回False
        if len(self.center_list) == 1:
            return False
        #注意比较向量相等要使用(x==y).all()
        elif ((self.center_list[-2].z0 == self.center_list[-1].z0).all() and 
            (self.center_list[-2].z1 == self.center_list[-1].z1).all() and 
            (self.center_list[-2].z2 == self.center_list[-1].z2).all()):
            #若两次的中心向量都相同，停止计算
            return True
        else:
            return False

## 进行训练

In [61]:
x = Kmeans_for_iris(data=data,list0=list_class0,list1=list_class1,
           list2=list_class2,center_list=center_list)

#定义计算次数
counter = 0

while x.stop_compute()!=True:
    print("当前聚类中心为:\n","第一类:",x.center_list[-1].z0,"\n第二类:",x.center_list[-1].z1,"\n第三类:",x.center_list[-1].z2)
    x.data_classify()
    x.compute_center_vector()
    counter+=1

print("\n--------迭代计算次数为:",counter)

print(len(x.list0),len(x.list1),len(x.list2))

当前聚类中心为:
 第一类: [5.1 3.5 1.4 0.2] 
第二类: [4.9 3.  1.4 0.2] 
第三类: [4.7 3.2 1.3 0.2]
当前聚类中心为:
 第一类: [[6.17727273 3.25795455 4.05568182 1.34545455]] 
第二类: [[5.54509804 2.67058824 3.76470588 1.16078431]] 
第三类: [[4.55454545 3.2        1.35454545 0.2       ]]
当前聚类中心为:
 第一类: [[6.52297297 2.97297297 5.24189189 1.83783784]] 
第二类: [[5.51923077 2.58461538 3.95       1.21538462]] 
第三类: [[5.006 3.418 1.464 0.244]]
当前聚类中心为:
 第一类: [[6.57014925 2.9880597  5.33880597 1.88507463]] 
第二类: [[5.63636364 2.63636364 4.02727273 1.25151515]] 
第三类: [[5.006 3.418 1.464 0.244]]
当前聚类中心为:
 第一类: [[6.6015873  2.98571429 5.38412698 1.91587302]] 
第二类: [[5.68378378 2.67837838 4.09189189 1.26756757]] 
第三类: [[5.006 3.418 1.464 0.244]]
当前聚类中心为:
 第一类: [[6.63220339 2.99830508 5.43050847 1.93728814]] 
第二类: [[5.72926829 2.6902439  4.15121951 1.3       ]] 
第三类: [[5.006 3.418 1.464 0.244]]
当前聚类中心为:
 第一类: [[6.66481481 3.00740741 5.5        1.96851852]] 
第二类: [[5.78913043 2.71304348 4.20869565 1.3326087 ]] 
第三类: [[5.006 3.418 1.464 0

## 对结果进行评测

In [62]:
x.list0

[array([7. , 3.2, 4.7, 1.4, 1. ]),
 array([6.9, 3.1, 4.9, 1.5, 1. ]),
 array([6.7, 3. , 5. , 1.7, 1. ]),
 array([6.3, 3.3, 6. , 2.5, 2. ]),
 array([7.1, 3. , 5.9, 2.1, 2. ]),
 array([6.3, 2.9, 5.6, 1.8, 2. ]),
 array([6.5, 3. , 5.8, 2.2, 2. ]),
 array([7.6, 3. , 6.6, 2.1, 2. ]),
 array([7.3, 2.9, 6.3, 1.8, 2. ]),
 array([6.7, 2.5, 5.8, 1.8, 2. ]),
 array([7.2, 3.6, 6.1, 2.5, 2. ]),
 array([6.5, 3.2, 5.1, 2. , 2. ]),
 array([6.4, 2.7, 5.3, 1.9, 2. ]),
 array([6.8, 3. , 5.5, 2.1, 2. ]),
 array([6.4, 3.2, 5.3, 2.3, 2. ]),
 array([6.5, 3. , 5.5, 1.8, 2. ]),
 array([7.7, 3.8, 6.7, 2.2, 2. ]),
 array([7.7, 2.6, 6.9, 2.3, 2. ]),
 array([6.9, 3.2, 5.7, 2.3, 2. ]),
 array([7.7, 2.8, 6.7, 2. , 2. ]),
 array([6.7, 3.3, 5.7, 2.1, 2. ]),
 array([7.2, 3.2, 6. , 1.8, 2. ]),
 array([6.4, 2.8, 5.6, 2.1, 2. ]),
 array([7.2, 3. , 5.8, 1.6, 2. ]),
 array([7.4, 2.8, 6.1, 1.9, 2. ]),
 array([7.9, 3.8, 6.4, 2. , 2. ]),
 array([6.4, 2.8, 5.6, 2.2, 2. ]),
 array([6.1, 2.6, 5.6, 1.4, 2. ]),
 array([7.7, 3. , 6.

In [63]:
x.list1

[array([6.4, 3.2, 4.5, 1.5, 1. ]),
 array([5.5, 2.3, 4. , 1.3, 1. ]),
 array([6.5, 2.8, 4.6, 1.5, 1. ]),
 array([5.7, 2.8, 4.5, 1.3, 1. ]),
 array([6.3, 3.3, 4.7, 1.6, 1. ]),
 array([4.9, 2.4, 3.3, 1. , 1. ]),
 array([6.6, 2.9, 4.6, 1.3, 1. ]),
 array([5.2, 2.7, 3.9, 1.4, 1. ]),
 array([5. , 2. , 3.5, 1. , 1. ]),
 array([5.9, 3. , 4.2, 1.5, 1. ]),
 array([6. , 2.2, 4. , 1. , 1. ]),
 array([6.1, 2.9, 4.7, 1.4, 1. ]),
 array([5.6, 2.9, 3.6, 1.3, 1. ]),
 array([6.7, 3.1, 4.4, 1.4, 1. ]),
 array([5.6, 3. , 4.5, 1.5, 1. ]),
 array([5.8, 2.7, 4.1, 1. , 1. ]),
 array([6.2, 2.2, 4.5, 1.5, 1. ]),
 array([5.6, 2.5, 3.9, 1.1, 1. ]),
 array([5.9, 3.2, 4.8, 1.8, 1. ]),
 array([6.1, 2.8, 4. , 1.3, 1. ]),
 array([6.3, 2.5, 4.9, 1.5, 1. ]),
 array([6.1, 2.8, 4.7, 1.2, 1. ]),
 array([6.4, 2.9, 4.3, 1.3, 1. ]),
 array([6.6, 3. , 4.4, 1.4, 1. ]),
 array([6.8, 2.8, 4.8, 1.4, 1. ]),
 array([6. , 2.9, 4.5, 1.5, 1. ]),
 array([5.7, 2.6, 3.5, 1. , 1. ]),
 array([5.5, 2.4, 3.8, 1.1, 1. ]),
 array([5.5, 2.4, 3.

In [64]:
x.list2

[array([5.1, 3.5, 1.4, 0.2, 0. ]),
 array([4.9, 3. , 1.4, 0.2, 0. ]),
 array([4.7, 3.2, 1.3, 0.2, 0. ]),
 array([4.6, 3.1, 1.5, 0.2, 0. ]),
 array([5. , 3.6, 1.4, 0.2, 0. ]),
 array([5.4, 3.9, 1.7, 0.4, 0. ]),
 array([4.6, 3.4, 1.4, 0.3, 0. ]),
 array([5. , 3.4, 1.5, 0.2, 0. ]),
 array([4.4, 2.9, 1.4, 0.2, 0. ]),
 array([4.9, 3.1, 1.5, 0.1, 0. ]),
 array([5.4, 3.7, 1.5, 0.2, 0. ]),
 array([4.8, 3.4, 1.6, 0.2, 0. ]),
 array([4.8, 3. , 1.4, 0.1, 0. ]),
 array([4.3, 3. , 1.1, 0.1, 0. ]),
 array([5.8, 4. , 1.2, 0.2, 0. ]),
 array([5.7, 4.4, 1.5, 0.4, 0. ]),
 array([5.4, 3.9, 1.3, 0.4, 0. ]),
 array([5.1, 3.5, 1.4, 0.3, 0. ]),
 array([5.7, 3.8, 1.7, 0.3, 0. ]),
 array([5.1, 3.8, 1.5, 0.3, 0. ]),
 array([5.4, 3.4, 1.7, 0.2, 0. ]),
 array([5.1, 3.7, 1.5, 0.4, 0. ]),
 array([4.6, 3.6, 1. , 0.2, 0. ]),
 array([5.1, 3.3, 1.7, 0.5, 0. ]),
 array([4.8, 3.4, 1.9, 0.2, 0. ]),
 array([5. , 3. , 1.6, 0.2, 0. ]),
 array([5. , 3.4, 1.6, 0.4, 0. ]),
 array([5.2, 3.5, 1.5, 0.2, 0. ]),
 array([5.2, 3.4, 1.

### 有一点麻烦的是分完类后，并不知道这是第几类，必须打印出来才能知道是
### 第几类，或者借助统计类别标签，多数者为类别数

### 统计列表中的类别来获取对应的标签

In [65]:
def statistic(_list):
    """
        统计类别以获得标签
    """
    c0 = 0
    c1 = 0
    c2 = 0
    for sample in _list:
        foo = sample[4:5]
        #print(foo)
        if foo==0:
            c0+=1
        elif foo==1:
            c1+=1
        else:
            c2+=1
    #print(c0,c1,c2)
    #Voting
    if c0 > c1 and c0 > c2: 
        return 0
    if c1 > c0 and c1 > c2: 
        return 1
    if c2 > c1 and c2 > c0: 
        return 2

### 计算分类正确的个数

In [66]:
def correct_number(_list):
    """
        计算某一类的分类正确的个数
    """
    correct = 0
    label = statistic(_list)
    print("第",label,"类")
    for sample in _list:
        #使用sample[4:5]获取其标签
        if sample[4:5] == label:
            correct+=1
    return correct

In [67]:
correct0 = correct_number(x.list0)
print("正确个数:",correct0)

correct1 = correct_number(x.list1)
print("正确个数:",correct1)

correct2 = correct_number(x.list2)
print("正确个数:",correct2)


第 2 类
正确个数: 36
第 1 类
正确个数: 47
第 0 类
正确个数: 50


## 计算指标

In [68]:
OA = (correct0+correct1+correct2)/x.data.shape[0]
print('Overall Accuracy:',OA*100,'%')

AA_0 = correct0/50
AA_1 = correct1/50
AA_2 = correct2/50
AA = (AA_0+AA_1+AA_2)/3

print('Average Accuracy:',AA*100,'%')

Overall Accuracy: 88.66666666666667 %
Average Accuracy: 88.66666666666667 %
